In [1]:
from google.colab import files
uploaded = files.upload()

Saving RegresiUTSTelkom.csv to RegresiUTSTelkom.csv


In [2]:
# Import library yang dibutuhkan
import pandas as pd

# Membaca data dari file yang diupload
df = pd.read_csv('/content/RegresiUTSTelkom.csv')

# Menampilkan nama kolom untuk memeriksa
print("Nama kolom dalam dataset:")
print(df.columns)

# Menampilkan 5 baris pertama dari data untuk pemeriksaan
df.head()

# Mengecek apakah ada nilai yang hilang
print("\nJumlah nilai yang hilang per kolom:")
print(df.isnull().sum())

# Menghapus baris dengan nilai yang hilang
df_cleaned = df.dropna()

# Mengecek apakah ada duplikat dalam data
df_cleaned = df_cleaned.drop_duplicates()

# Menampilkan statistik deskriptif dari data yang telah dibersihkan
print("\nStatistik Deskriptif Data:")
print(df_cleaned.describe())

Nama kolom dalam dataset:
Index(['2001', '49.94357', '21.47114', '73.0775', '8.74861', '-17.40628',
       '-13.09905', '-25.01202', '-12.23257', '7.83089', '-2.46783', '3.32136',
       '-2.31521', '10.20556', '611.10913', '951.0896', '698.11428',
       '408.98485', '383.70912', '326.51512', '238.11327', '251.42414',
       '187.17351', '100.42652', '179.19498', '-8.41558', '-317.87038',
       '95.86266', '48.10259', '-95.66303', '-18.06215', '1.96984', '34.42438',
       '11.7267', '1.3679', '7.79444', '-0.36994', '-133.67852', '-83.26165',
       '-37.29765', '73.04667', '-37.36684', '-3.13853', '-24.21531',
       '-13.23066', '15.93809', '-18.60478', '82.15479', '240.5798',
       '-10.29407', '31.58431', '-25.38187', '-3.90772', '13.29258', '41.5506',
       '-7.26272', '-21.00863', '105.50848', '64.29856', '26.08481',
       '-44.5911', '-8.30657', '7.93706', '-10.7366', '-95.44766', '-82.03307',
       '-35.59194', '4.69525', '70.95626', '28.09139', '6.02015', '-37.13767',
  

In [3]:
# Import library yang dibutuhkan untuk seleksi fitur
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_regression
import numpy as np
import pandas as pd

# Menggunakan kolom selain target untuk fitur X
X = df_cleaned.drop(columns='2001')  # '2001' sebagai kolom target
y = df_cleaned['2001']  # '2001' sebagai kolom target

# Menghapus fitur dengan variansi rendah
selector = VarianceThreshold(threshold=0.01)  # Variansi rendah akan dihapus
X_cleaned = selector.fit_transform(X)

# Menghitung korelasi antar fitur
corr_matrix = X.corr()

# Menghapus fitur yang memiliki korelasi lebih dari threshold (misalnya 0.9)
corr_matrix = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in corr_matrix.columns if any(corr_matrix[column].abs() > 0.9)]
X_cleaned = X.drop(columns=to_drop)

# Pilih fitur terbaik dengan menggunakan Mutual Information
mutual_info = mutual_info_regression(X_cleaned, y)
mutual_info = pd.Series(mutual_info, index=X_cleaned.columns)
mutual_info_sorted = mutual_info.sort_values(ascending=False)

# Pilih 10 fitur terbaik berdasarkan Mutual Information
X_selected = X_cleaned[mutual_info_sorted.head(10).index]

In [4]:
# Import library untuk preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Misalkan jika ada fitur kategorikal, lakukan encoding
# (Jika tidak ada fitur kategorikal, bisa dilewati)
if 'categorical_column' in df_cleaned.columns:
    le = LabelEncoder()
    df_cleaned['categorical_column'] = le.fit_transform(df_cleaned['categorical_column'])

# Lakukan normalisasi pada fitur numerik terpilih
scaler = StandardScaler()
X_selected_scaled = scaler.fit_transform(X_selected)

# Membagi data menjadi data latih dan data uji
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_selected_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# Import model regresi
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline  # <--- Tambahkan ini
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# List model yang akan digunakan
models = {
    'Linear Regression': LinearRegression(),
    'Polynomial Regression': Pipeline([('poly', PolynomialFeatures(degree=2)), ('linear', LinearRegression())]),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'K-Nearest Neighbors Regressor': KNeighborsRegressor(),
    'Bagging Regressor': BaggingRegressor(),
    'AdaBoost Regressor': AdaBoostRegressor(),
    'Gradient Boosting Regressor': GradientBoostingRegressor(),
    'SVR': SVR()
}

# Melatih dan mengevaluasi model
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Evaluasi menggunakan metrik
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    # Simpan hasil evaluasi
    results[name] = {'RMSE': rmse, 'MSE': mse, 'R^2': r2}

# Tampilkan hasil evaluasi
results_df = pd.DataFrame(results).T
print("\nEvaluasi Model:")
print(results_df)